In [ ]:
import pandas as pd
from bacGWAStatLearn.TreeTune.ClassTuning import Tuning
from bacGWAStatLearn.FeatureSelect.ClassFeatureSelection import FeatSelection
import os
import pickle
from joblib import Parallel, delayed

: 

In [ ]:
parentDIR = os.getcwd()
numCores = -1#os.cpu_count()
print(os.cpu_count(), numCores)
TuneObj = Tuning()
X, Y, n_splits, class_weight =TuneObj.XY_index(X='/Users/paulmacos/Desktop/GWASgit/example/BorutaOUT_1.00.05_RF/BorutaOUT/RF_FinalFeatureMatrix_BorutaSelected_Sim1.txt', Y='/Users/paulmacos/Desktop/GWASgit/example/BalancedArbitrary_Response_1.txt', BacGWASim=False)
# RAND=np.random.RandomState(4)
print((X.shape))
print((Y.shape))
print((Y.value_counts()))

In [ ]:
with open('/Users/paulmacos/Desktop/GWASgit/example/BorutaOUT_1.00.05_RF/TuningOUT_RF/RFOptimal_Tuning2_Sim1.pickle', 'rb') as filehandle:
    RFoptimal=pickle.load(filehandle)
# RFoptimal = pickle.load('TuningOUT/RFOptimal_Tuning2_'+options.sim+'.pickle')


# RankedFeats.sort_values(0, ascending=False, key=abs, inplace=True)

# if not os.path.isdir('FinalFeatures_RF'):
#     os.makedirs("FinalFeatures_RF")
# os.chdir("FinalFeatures_RF")
NewNASPobj = FeatSelection(repeat=4)



In [ ]:
FI = Parallel(n_jobs=numCores,  backend="loky", prefer='processes' )(delayed(NewNASPobj.FeatImp)(RFoptimal, X, Y, rand=i,) for i in range(NewNASPobj.repeat))
FI = pd.concat(FI)
FI=FI.reset_index().groupby('index', as_index=True).mean().sort_values(0,ascending=False)
FI.index.rename('variant', inplace=True)
FI.to_csv('FeatureImportance_Ranked_Sim1.out', sep='\t')

# SHAPdata =  Parallel(n_jobs=numCores, backend='loky', prefer='processes' )(delayed(NewNASPobj.SHAPscores)(RFoptimal, X, Y,rand=i,)for i in range(NewNASPobj.repeat))
# SHAPster = pd.concat(SHAPdata).groupby('variant', as_index=False).mean().sort_values('mean', key=abs, ascending=False).reset_index(drop=True)
# SHAPster.to_csv('SHAPster_Ranked_Sim1.out', sep='\t')


PI = Parallel(n_jobs=numCores, backend="loky", prefer='processes' )(delayed(NewNASPobj.PermImp)(RFoptimal, X, Y,5, scorer='balanced_accuracy', rand=i, ) for i in range(NewNASPobj.repeat))
PI = pd.concat(PI)
PI=PI.reset_index().groupby('index', as_index=True).mean().sort_values(0,ascending=False)
PI.index.rename('variant', inplace=True)
PI.to_csv('PermutationImportance_Ranked_Sim1.out', sep='\t')